In [ ]:
import os, time, shutil, tarfile
from minio import Minio
from tqdm import tnrange

testList = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
runTimeListSmallFile = []
runTimeListBigFile = []

shutil.rmtree('minio_test_data')
os.mkdir('minio_test_data')

for size in testList:
    os.mkdir('minio_test_data/'+str(size))
    tar=tarfile.open('minio_test_data/'+str(size)+'.tar','w')  
    for i in range(size):
        shutil.copyfile('m_7.jpg', 'minio_test_data/'+str(size)+'/'+str(i)+'.jpg')
        tar.add('minio_test_data/'+str(size)+'/'+str(i)+'.jpg')
    tar.close()

    minioClient = Minio('127.0.0.1:30648',
                        access_key='wei840222',
                        secret_key='j04rmp4jo3',
                        secure=False)
    
    minioClient.make_bucket('test-'+str(size), location="us-east-1")
    with tnrange(size) as t:
        start = time.time()
        for i in t:
            minioClient.fput_object('test-'+str(size), str(i)+'.jpg', 'minio_test_data/'+str(size)+'/'+str(i)+'.jpg')
        end = time.time()
        runTimeListSmallFile.append(end-start)
        start = time.time()
        minioClient.fput_object('test-'+str(size), str(size)+'.tar', 'minio_test_data/'+str(size)+'.tar')
        end = time.time()
        runTimeListBigFile.append(end-start)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
fig = plt.figure()
plt.xlabel('files (4.33MB per file)')
plt.ylabel('time (sec)')
plt.plot(testList, runTimeListSmallFile, 'o',
         testList, runTimeListSmallFile, '--',
         testList, runTimeListBigFile, 'x',
         testList, runTimeListBigFile, '--')